# Tensorflow Adversarial Embedding MNIST demo for Securing AI Lab deployment

This demo will cover the adversarial clean label backdoor attack on an MNIST-LeNet model.

The following two sections cover experiment setup and is similar across all demos.

To access demo results in MlFlow, please follow the general experiment setup steps outlined in `basic-mlflow-demo`.

## Setup: Experiment Name and MNIST Dataset

Here we will import the necessary Python modules and ensure the proper environment variables are set so that all the code blocks will work as expected.

### Important: Users will need to verify or update the following parameters:
- Ensure that the `USERNAME` parameter is set to your own name.
- Ensure that the `DATASET_DIR` parameter is set to the location of the MNIST dataset directory. Currently set to `/nfs/data/Mnist` as the default location.
- (Optional) Set the `EXPERIMENT_NAME` parameter to your own preferred experiment name.

Other parameters can be modified to alter the RESTful API and MLFlow tracking addresses. 

In [1]:
# Import packages from the Python standard library
import os
import pprint
import time
import warnings
from pathlib import Path
from typing import Tuple
import tarfile

# Please enter your username here.
USERNAME = "howard"

# Ensure that the MNIST dataset location is properly set here.
DATASET_DIR = "/nfs/data/Mnist"

# Filter out warning messages
warnings.filterwarnings("ignore")

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = "http://localhost:30080"

# Base API address
RESTAPI_API_BASE = f"{RESTAPI_ADDRESS}/api"

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = "http://localhost:35000"

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Experiment name (note the username_ prefix convention)
EXPERIMENT_NAME = f"{USERNAME}_mnist_clean_label_backdoor"

# Set MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
if os.getenv("MLFLOW_TRACKING_URI") is None:
    os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Import third-party Python packages
import numpy as np
import requests
from mlflow.tracking import MlflowClient

# Import utils.py file
import utils

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

Check that the Makefile works in your environment by executing the `bash` code block below,

In [2]:
%%bash

# Running this will just list the available rules defined in the demo's Makefile.
make

Available rules:

clean               Remove temporary files 
data                Download and prepare MNIST dataset 
initdb              Initialize the RESTful API database 
services            Launch the Minio S3 and MLFlow Tracking services 
teardown            Destroy service containers 
workflows           Create workflows tarball 


## Submit and run jobs

The jobs that we will be running are implemented in the Python source files under `src/`, which will be executed using the entrypoints defined in the `MLproject` file.
To get this information into the architecture, we need to package those files up into an archive and upload it to the lab API.
For convenience, the `Makefile` provides a rule for creating the archive file, just run `make workflows`,

In [3]:
%%bash

# Create the workflows.tar.gz file
make workflows

make: Nothing to be done for 'workflows'.


To connect with the endpoint, we will use a client class defined in the `utils.py` file that is able to connect with the lab's RESTful API using the HTTP protocol.
We connect using the client below,

In [4]:
restapi_client = utils.SecuringAIClient(address=RESTAPI_API_BASE)

We need to register an experiment under which to collect our job runs.
The code below checks if the relevant experiment exists.
If it does, then it just returns info about the experiment, if it doesn't, it then registers the new experiment.

In [5]:
response_experiment = restapi_client.get_experiment_by_name(name=EXPERIMENT_NAME)

if response_experiment is None or "Not Found" in response_experiment.get("message", []):
    response_experiment = restapi_client.register_experiment(name=EXPERIMENT_NAME)

response_experiment

{'createdOn': '2021-01-25T20:03:56.964324',
 'experimentId': 22,
 'lastModified': '2021-01-25T20:03:56.964324',
 'name': 'howard_mnist_clean_label_backdoor'}

In [6]:
# The following helper functions will recheck the job responses until the job is completed or a run ID is available. 
# The run ID is needed to link dependencies between jobs.
def mlflow_run_id_is_not_known(job_response):
    return job_response["mlflowRunId"] is None and job_response["status"] not in [
        "failed",
        "finished",
    ]

def get_run_id(job_response):
    while mlflow_run_id_is_not_known(job_response):
        time.sleep(1)
        job_response = restapi_client.get_job_by_id(job_response["jobId"])
        
    return job_response

def wait_until_finished(job_response):
    # First make sure job has started.
    job_response = get_run_id(job_response)
    
    # Next re-check job until it has stopped running.
    while (job_response["status"] not in ["failed", "finished"]):
        time.sleep(1)
        job_response = restapi_client.get_job_by_id(job_response["jobId"])
    
    return job_response

# Helper function for viewing MLflow results.
def get_mlflow_results(job_response):
    mlflow_client = MlflowClient()
    job_response = wait_until_finished(job_response)
    
    if(job_response['status']=="failed"):
        return {}
    
    run = mlflow_client.get_run(job_response["mlflowRunId"])  
    
    while(len(run.data.metrics) == 0):
        time.sleep(1)
        run = mlflow_client.get_run(job_response["mlflowRunId"])
        
    return run

def print_mlflow_results(response):
    results = get_mlflow_results(response)
    pprint.pprint(results.data.metrics)

## MNIST Training: Baseline Model

Next, we need to train our baseline model that will serve as a reference point for the effectiveness of our attacks.

We will be using the V100 GPUs that are available on the DGX Workstation, which we can use by submitting our job to the `"tensorflow_gpu"` queue.

In [7]:
def mlflow_run_id_is_not_known(response_fgm):
    return response_fgm["mlflowRunId"] is None and response_fgm["status"] not in [
        "failed",
        "finished",
    ]

response_le_net_train = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train",
    entry_point_kwargs=" ".join([
        "-P batch_size=256",
        "-P register_model=True",
        "-P model_architecture=le_net",
        "-P epochs=30",
        f"-P data_dir_train={DATASET_DIR}/training",
        f"-P data_dir_test={DATASET_DIR}/testing",
    ]),
    queue="tensorflow_gpu",
    timeout="1h",
    
)

print("Training job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_le_net_train)

response_le_net_train = get_run_id(response_le_net_train)
print_mlflow_results(response_le_net_train)

Training job for LeNet-5 neural network submitted

{'createdOn': '2021-02-15T08:03:26.699644',
 'dependsOn': None,
 'entryPoint': 'train',
 'entryPointKwargs': '-P batch_size=256 -P register_model=True -P '
                     'model_architecture=le_net -P epochs=30 -P '
                     'data_dir_train=/nfs/data/Mnist/training -P '
                     'data_dir_test=/nfs/data/Mnist/testing',
 'experimentId': 22,
 'jobId': '2e9eac60-c4ee-426e-83bb-da31d36b6ce3',
 'lastModified': '2021-02-15T08:03:26.699644',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '1h',
 'workflowUri': 's3://workflow/f8652e6d193549c4a54e4c0dad600d06/workflows.tar.gz'}
{'accuracy': 0.9827514290809631,
 'auc': 0.9993622899055481,
 'fn': 940.0,
 'fp': 706.0,
 'loss': 0.05485766302924903,
 'precision': 0.9852208495140076,
 'recall': 0.98041832447052,
 'restored_epoch': 4.0,
 'stopped_epoch': 9.0,
 'tn': 431330.0,
 'tp': 47064.0,
 'training_time_in_minutes': 6.9349874,
 'val_accuracy': 0.

In [ ]:
# Train a special model for making poisons

response_le_net_train_robust = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train",
    entry_point_kwargs=" ".join([
        "-P batch_size=256",
        "-P register_model=True",
        "-P model_architecture=le_net",
        "-P model_tag=robust",
        "-P epochs=30",
        f"-P data_dir_train={DATASET_DIR}/training",
        f"-P data_dir_test={DATASET_DIR}/testing",
        "-P apply_defense=True",
    ]),
    queue="tensorflow_gpu",
    timeout="1h",
    
)

print("Training job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_le_net_train_robust)

response_le_net_train_robust = get_run_id(response_le_net_train_robust)
print_mlflow_results(response_le_net_train_robust)

Training job for LeNet-5 neural network submitted

{'createdOn': '2021-02-15T08:10:54.568068',
 'dependsOn': None,
 'entryPoint': 'train',
 'entryPointKwargs': '-P batch_size=256 -P register_model=True -P '
                     'model_architecture=le_net -P model_tag=robust -P '
                     'epochs=30 -P data_dir_train=/nfs/data/Mnist/training -P '
                     'data_dir_test=/nfs/data/Mnist/testing -P '
                     'apply_defense=True',
 'experimentId': 22,
 'jobId': '4eba9971-ada6-4c46-b879-b3c0ca6b7d89',
 'lastModified': '2021-02-15T08:10:54.568068',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '1h',
 'workflowUri': 's3://workflow/5a989dd31797453cb29c445e83318e79/workflows.tar.gz'}


## Generating Poisoned Images.

Now we will create our set of poisoned images.

Start by submitting the poison generation job below.

In [ ]:
## Create poisoned test images.
        
response_gen_poison_le_net_test = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="gen_poison_test_data",
    entry_point_kwargs=" ".join(
        [
            "-P model_architecture=le_net",
            f"-P data_dir={DATASET_DIR}/testing",
            "-P batch_size=100",
            "-P target_class=1",
            "-P poison_fraction=1",
            "-P label_type=test"
            
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_train["jobId"],
)

print("Backdoor poison attack (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_gen_poison_le_net_test)
print("")

response_gen_poison_le_net_test = get_run_id(response_gen_poison_le_net_test)

In [ ]:
## Create poisoned training images (clean_label).
        
response_gen_poison_le_net_train_clean = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="gen_poison_clean_data",
    entry_point_kwargs=" ".join(
        [
             f"-P model={EXPERIMENT_NAME}_robust_le_net/None",
            "-P model_architecture=le_net",
            f"-P data_dir={DATASET_DIR}/testing",
            "-P batch_size=200",
            "-P target_class=1",
            "-P poison_fraction=0.33",
            "-P label_type=train"
            
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_train["jobId"],
)

print("Backdoor poison attack (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_gen_poison_le_net_train_clean)
print("")

response_gen_poison_le_net_train_clean = get_run_id(response_gen_poison_le_net_train_clean)

# Generating Poisoned Model from Training Images

In [ ]:
# Now train a new model on the poisoned clean label images.
def mlflow_run_id_is_not_known(response_fgm):
    return response_fgm["mlflowRunId"] is None and response_fgm["status"] not in [
        "failed",
        "finished",
    ]

response_le_net_train_backdoor_model_clean = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train",
    entry_point_kwargs=" ".join([
        "-P batch_size=256",
        "-P register_model=True",
        "-P model_architecture=le_net",
        "-P epochs=30",
        "-P model_tag=data_poison",
        f"-P data_dir_train={DATASET_DIR}/training",
        f"-P data_dir_test={DATASET_DIR}/testing",
        "-P load_dataset_from_mlruns=true",
        f"-P training_dataset_run_id={response_gen_poison_le_net_train_clean['mlflowRunId']}",
    ]),
    depends_on=response_gen_poison_le_net_train_clean["jobId"],
    queue="tensorflow_gpu",
    timeout="1h",
    
)

print("Training job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_le_net_train_backdoor_model_clean)

response_le_net_train_backdoor_model_clean = get_run_id(response_le_net_train_backdoor_model_clean)
print_mlflow_results(response_le_net_train_backdoor_model_clean)

# Model Evaluation: Poisoned vs Regular Models on Backdoor-Poisoned Images.

Below we will compare the results of the regular model vs poisoned-backdoor model on backdoor test images.

In [ ]:
# Inference: Regular model trained on poisoned backdoor attack.

response_infer_pos_model_clean = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_gen_poison_le_net_test['mlflowRunId']}",
            f"-P model={EXPERIMENT_NAME}_data_poison_le_net/None",
            "-P batch_size=512",
            "-P model_architecture=le_net",
            "-P dataset_tar_name=adversarial_poison.tar.gz",
            "-P dataset_name=adv_poison_data",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_train_backdoor_model_clean["jobId"],
)

print("Inference job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_infer_pos_model_clean)
print_mlflow_results(response_infer_pos_model_clean)

In [ ]:
# Inference: Regular model on poisoned test images.

response_infer_reg_model = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_gen_poison_le_net_test['mlflowRunId']}",
            f"-P model={EXPERIMENT_NAME}_le_net/None",
            "-P batch_size=512",
            "-P model_architecture=le_net",
            "-P dataset_tar_name=adversarial_poison.tar.gz",
            "-P dataset_name=adv_poison_data",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_train["jobId"],
)

print("Inference job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_infer_reg_model)
print_mlflow_results(response_infer_reg_model)

# Now we will explore available defenses on the adversarial backdoor poisoning attack.

The following three jobs will run a selected defense (spatial smoothing, gaussian augmentation, or jpeg compression) and evaluate the defense on the baseline and backdoor trained models.

- The first job uses the selected defense entrypoint to apply a preprocessing defense over the poisoned test images.
- The second job runs the defended images against the poisoned backdoor model.
- The final job runs the defended images against the baseline model.

Ideally the defense will not impact the baseline model accuracy, while improving the backdoor model accuracy scores.

In [ ]:
defenses = ["gaussian_augmentation", "spatial_smoothing", "jpeg_compression"]
defense = defenses[2]
response_fgm_resnet50_attack_def = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point=defense,
    entry_point_kwargs=" ".join(
        [
            f"-P model={EXPERIMENT_NAME}_le_net/1",
            "-P model_architecture=le_net",
            f"-P data_dir={DATASET_DIR}/testing",
            "-P batch_size=20",
            "-P load_dataset_from_mlruns=true",
            f"-P dataset_run_id={response_gen_poison_le_net_test['mlflowRunId']}",
            "-P dataset_tar_name=adversarial_poison.tar.gz",
            "-P dataset_name=adv_poison_data",
            
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_gen_poison_le_net_test["jobId"],
)


print(f"FGM {defense} defense (LeNet architecture) job submitted")
print("")
pprint.pprint(response_fgm_resnet50_attack_def)
print("")

response_fgm_resnet50_attack_def = get_run_id(response_fgm_resnet50_attack_def)

In [ ]:
# Inference: Poisoned model on poisoned test images.

response_infer_pos_model = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_fgm_resnet50_attack_def['mlflowRunId']}",
             f"-P model={EXPERIMENT_NAME}_data_poison_le_net/None",
            "-P batch_size=512",
            "-P model_architecture=le_net",
            f"-P dataset_tar_name={defense}_dataset.tar.gz",
            "-P dataset_name=adv_testing",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_fgm_resnet50_attack_def["jobId"],
)

print("Inference job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_infer_pos_model)
print_mlflow_results(response_infer_pos_model)

In [ ]:
# Inference: Regular model on poisoned test images.
response_infer_reg_model = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_fgm_resnet50_attack_def['mlflowRunId']}",
            f"-P model={EXPERIMENT_NAME}_le_net/None",
            "-P batch_size=512",
            "-P model_architecture=le_net",
            f"-P dataset_tar_name={defense}_dataset.tar.gz",
            "-P dataset_name=adv_testing",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_fgm_resnet50_attack_def["jobId"],
)

print("Inference job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_infer_reg_model)
print_mlflow_results(response_infer_reg_model)

<a id='querying_cell'></a>
## Querying the MLFlow Tracking Service

Currently the lab API can only be used to register experiments and start jobs, so if users wish to extract their results programmatically, they can use the `MlflowClient()` class from the `mlflow` Python package to connect and query their results.
Since we captured the run ids generated by MLFlow, we can easily retrieve the data logged about one of our jobs and inspect the results.
To start the client, we simply need to run,

In [22]:
mlflow_client = MlflowClient()

The client uses the environment variable `MLFLOW_TRACKING_URI` to figure out how to connect to the MLFlow Tracking Service, which we configured near the top of this notebook.
To query the results of one of our runs, we just need to pass the run id to the client's `get_run()` method.
As an example, let's query the run results for the patch attack applied to the LeNet-5 architecture,

In [23]:
run_le_net = mlflow_client.get_run(response_le_net_train["mlflowRunId"])

If the request completed successfully, we should now be able to query data collected during the run.
For example, to review the collected metrics, we just use,

In [24]:
pprint.pprint(run_le_net.data.metrics)

{'accuracy': 0.9836680293083191,
 'auc': 0.9994320869445801,
 'fn': 901.0,
 'fp': 666.0,
 'loss': 0.053702861484105704,
 'precision': 0.9860578775405884,
 'recall': 0.9812307357788086,
 'restored_epoch': 4.0,
 'stopped_epoch': 9.0,
 'tn': 431370.0,
 'tp': 47103.0,
 'training_time_in_minutes': 7.423224333333334,
 'val_accuracy': 0.9872457385063171,
 'val_auc': 0.9992121458053589,
 'val_fn': 162.0,
 'val_fp': 143.0,
 'val_loss': 0.043615993835586814,
 'val_precision': 0.988060474395752,
 'val_recall': 0.9864954948425293,
 'val_tn': 107821.0,
 'val_tp': 11834.0}


To review the run's parameters, we use,

In [25]:
pprint.pprint(run_le_net.data.params)

{'apply_defense': 'False',
 'baseline': 'None',
 'batch_size': '256',
 'data_dir_test': '/nfs/data/Mnist/testing',
 'data_dir_train': '/nfs/data/Mnist/training',
 'dataset_name': 'adv_poison_data',
 'dataset_seed': '491391921',
 'dataset_tar_name': 'adversarial_poison.tar.gz',
 'entry_point_seed': '9362298795793007086857596422728629728',
 'epochs': '30',
 'learning_rate': '0.001',
 'load_dataset_from_mlruns': 'False',
 'min_delta': '-0.01',
 'model_architecture': 'le_net',
 'model_tag': "''",
 'monitor': 'val_loss',
 'opt_amsgrad': 'False',
 'opt_beta_1': '0.9',
 'opt_beta_2': '0.999',
 'opt_decay': '0.0',
 'opt_epsilon': '1e-07',
 'opt_learning_rate': '0.001',
 'opt_name': 'Adam',
 'optimizer': 'adam',
 'patience': '5',
 'register_model': 'True',
 'restore_best_weights': 'True',
 'seed': '-1',
 'tensorflow_global_seed': '878021404',
 'training_dataset_run_id': 'None',
 'validation_split': '0.2'}


To review the run's tags, we use,

In [26]:
pprint.pprint(run_le_net.data.tags)

{'mlflow.log-model.history': '[{"run_id": "ce1bb519957c4c62aa87fea44b86ed61", '
                             '"artifact_path": "model", "utc_time_created": '
                             '"2021-02-15 06:24:08.566942", "flavors": '
                             '{"keras": {"keras_module": '
                             '"tensorflow_core.keras", "keras_version": '
                             '"2.2.4-tf", "data": "data"}, "python_function": '
                             '{"loader_module": "mlflow.keras", '
                             '"python_version": "3.7.9", "data": "data", '
                             '"env": "conda.yaml"}}}]',
 'mlflow.project.backend': 'securingai',
 'mlflow.project.entryPoint': 'train',
 'mlflow.source.name': '/work/tmp5zfs9qj5',
 'mlflow.source.type': 'PROJECT',
 'mlflow.user': 'securingai',
 'securingai.dependsOn': 'None',
 'securingai.jobId': '50bcd46f-4f68-4866-9e1b-520697cebe84',
 'securingai.queue': 'tensorflow_gpu'}


There are many things you can query using the MLFlow client.
[The MLFlow documentation gives a full overview of the methods that are available](https://www.mlflow.org/docs/latest/python_api/mlflow.tracking.html#mlflow.tracking.MlflowClient).